<a href="https://colab.research.google.com/github/DucBox/CyberSecurity/blob/main/Salsa20_%2B_DES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def rotl(value, shift, bits=32):
    return ((value << shift) | (value >> (bits - shift))) & ((1 << bits) - 1)

def quarter_round(a, b, c, d):
    a = (a + b) & 0xffffffff; d ^= a; d = rotl(d, 16)
    c = (c + d) & 0xffffffff; b ^= c; b = rotl(b, 12)
    a = (a + b) & 0xffffffff; d ^= a; d = rotl(d, 8)
    c = (c + d) & 0xffffffff; b ^= c; b = rotl(b, 7)
    return a, b, c, d

def salsa20_hash(state):
    x = list(state)
    for _ in range(10):  # 20 rounds, performed in pairs
        # Column rounds
        x[4], x[8], x[12], x[0] = quarter_round(x[4], x[8], x[12], x[0])
        x[9], x[13], x[1], x[5] = quarter_round(x[9], x[13], x[1], x[5])
        x[14], x[2], x[6], x[10] = quarter_round(x[14], x[2], x[6], x[10])
        x[3], x[7], x[11], x[15] = quarter_round(x[3], x[7], x[11], x[15])
        # Diagonal rounds
        x[1], x[2], x[3], x[0] = quarter_round(x[1], x[2], x[3], x[0])
        x[6], x[7], x[4], x[5] = quarter_round(x[6], x[7], x[4], x[5])
        x[11], x[8], x[9], x[10] = quarter_round(x[11], x[8], x[9], x[10])
        x[12], x[13], x[14], x[15] = quarter_round(x[12], x[13], x[14], x[15])
    return [(x[i] + state[i]) & 0xffffffff for i in range(16)]

def salsa20_key_schedule(key, nonce, counter):
    constants = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]
    key_schedule = constants[:1] + [int.from_bytes(key[i:i+4], 'little') for i in range(0, 16, 4)] \
                  + constants[1:3] + [counter & 0xffffffff, counter >> 32] \
                  + [int.from_bytes(nonce[i:i+4], 'little') for i in range(0, 8, 4)] + constants[3:]
    if len(key) == 32:
        key_schedule[1:5] += [int.from_bytes(key[i:i+4], 'little') for i in range(16, 32, 4)]
    elif len(key) != 16:
        raise ValueError("Key must be either 16 or 32 bytes long.")
    return key_schedule
def encrypt(plaintext, key, nonce):
    assert len(nonce) == 8, "Nonce must be 8 bytes long"
    encrypted = b''
    counter = 0
    while plaintext:
        block = plaintext[:64]
        plaintext = plaintext[64:]
        key_schedule = salsa20_key_schedule(key, nonce, counter)
        keystream = salsa20_hash(key_schedule)
        keystream_bytes = b''.join(word.to_bytes(4, 'little') for word in keystream)
        encrypted_block = bytes(a ^ b for a, b in zip(block, keystream_bytes[:len(block)]))
        encrypted += encrypted_block
        counter += 1
    return encrypted

def decrypt(ciphertext, key, nonce):
    return encrypt(ciphertext, key, nonce)

# Example usage
key = b"this_is_a_32_byte_key_for_salsa!"
nonce = b"nonce123"
plaintext = b"Hello, Salsa20!"

encrypted = encrypt(plaintext, key, nonce)
print(f"Encrypted: {encrypted.hex()}")

decrypted = decrypt(encrypted, key, nonce)
print(f"Decrypted: {decrypted.decode()}")



Encrypted: 0109fc750bc2b016264d845aedb744
Decrypted: Hello, Salsa20!


In [ ]:
def KSA(key):
    key_length = len(key)
    S = list(range(256))
    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % key_length]) % 256
        S[i], S[j] = S[j], S[i]
    return S

def PRGA(S):
    i = 0
    j = 0
    while True:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        K = S[(S[i] + S[j]) % 256]
        yield K

def RC4(key):
    S = KSA(key)
    return PRGA(S)


In [ ]:
def benchmark_stream_generation(cipher_func, key, nonce=None, data_size=1024*1024):
    start_time = time.time()
    # Generate a block of data
    data = b'\x00' * data_size
    if nonce:  # For Salsa20
        _ = cipher_func(data, key, nonce)
    else:  # For RC4, assuming cipher_func is adjusted to return bytes
        _ = cipher_func(data, key)
    end_time = time.time()
    return end_time - start_time

# Adjust RC4 to return bytes instead of a generator
def RC4_to_bytes(key, data):
    keystream = RC4(key)
    return bytes([next(keystream) for _ in range(len(data))])

# Benchmark Salsa20 and RC4
salsa20_time = benchmark_stream_generation(encrypt, key, nonce)
rc4_time = benchmark_stream_generation(RC4_to_bytes, key)

print(f"Salsa20 Generation Time: {salsa20_time} seconds")
print(f"RC4 Generation Time: {rc4_time} seconds")


Salsa20 Generation Time: 5.387058973312378 seconds
RC4 Generation Time: 0.0003523826599121094 seconds


In [ ]:
def key_sensitivity_test(encrypt_function, key, nonce=None):
    data = b'\x00' * 512  # Dùng một khối dữ liệu nhỏ để mã hóa
    flipped_key = bytearray(key)
    flipped_key[0] ^= 1  # Flip the first bit of the key

    encrypted1 = encrypt_function(data, key, nonce) if nonce else RC4_to_bytes(key, data)
    encrypted2 = encrypt_function(data, bytes(flipped_key), nonce) if nonce else RC4_to_bytes(bytes(flipped_key), data)

    # Calculate the number of differing bytes
    differences = sum(1 for x, y in zip(encrypted1, encrypted2) if x != y)
    return differences

# So sánh độ nhạy khóa
difference_salsa20 = key_sensitivity_test(encrypt, key, nonce)
difference_rc4 = key_sensitivity_test(RC4_to_bytes, key)

print(f"Salsa20 Key Sensitivity: {difference_salsa20} bytes differ")
print(f"RC4 Key Sensitivity: {difference_rc4} bytes differ")


Salsa20 Key Sensitivity: 511 bytes differ
RC4 Key Sensitivity: 510 bytes differ


In [ ]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00


In [ ]:
from Crypto.Cipher import DES, ARC4
from Crypto.Random import get_random_bytes
import time

# Padding để đảm bảo dữ liệu có độ dài là bội số của DES block size
def pad(text):
    while len(text) % 8 != 0:
        text += b' '
    return text

# Hàm mã hóa DES
def encrypt_des(plaintext, key):
    des = DES.new(key, DES.MODE_ECB)
    padded_text = pad(plaintext)
    encrypted_text = des.encrypt(padded_text)
    return encrypted_text

# Hàm giải mã DES
def decrypt_des(ciphertext, key):
    des = DES.new(key, DES.MODE_ECB)
    decrypted_text = des.decrypt(ciphertext)
    return decrypted_text.strip()

# Hàm mã hóa RC4
def encrypt_rc4(plaintext, key):
    rc4 = ARC4.new(key)
    encrypted_text = rc4.encrypt(plaintext)
    return encrypted_text

# Hàm giải mã RC4
def decrypt_rc4(ciphertext, key):
    rc4 = ARC4.new(key)
    decrypted_text = rc4.decrypt(ciphertext)
    return decrypted_text

# Đo lường thời gian
def measure_time(func, *args):
    start_time = time.time()
    func(*args)
    end_time = time.time()
    return end_time - start_time

# Tạo key ngẫu nhiên cho DES và RC4
# key_des = get_random_bytes(8)
key_rc4 = get_random_bytes (16)

key_des = b"mysecret"

# Plaintext mẫu
plaintext = b"Hello, DES!"

# Mã hóa DES
encrypted_des = encrypt_des(plaintext, key_des)
print("Encrypted Text (DES):", encrypted_des)
print("Encrypted Text (DES-hex):", encrypted_des.hex())

# Giải mã DES
decrypted_des = decrypt_des(encrypted_des, key_des)
print("Decrypted Text (DES):", decrypted_des.decode())


Encrypted Text (DES): b'\x90\xe3\x9d\x8de\xf1\xaa\xe6\xd5lo\xbb\xf7\xc68\xcf'
Encrypted Text (DES-hex): 90e39d8d65f1aae6d56c6fbbf7c638cf
Decrypted Text (DES): Hello, DES!


In [ ]:

# Tạo các case đa dạng để test
cases = [
    b"Short text.",
    b"https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4",
    b"https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4 https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4 It contains more characters and therefore, it is expected to take a longer time to encrypt and decrypt compared to shorter texts.",
    b"The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog"
]

# Đo lường thời gian mã hóa cho mỗi case
for i, data in enumerate(cases, 1):
    time_des = measure_time(encrypt_des, data, key_des)
    time_rc4 = measure_time(encrypt_rc4, data, key_rc4)
    print(f'Case {i}:')
    print(f'Time taken for DES encryption: {time_des:.6f} seconds')
    print(f'Time taken for RC4 encryption: {time_rc4:.6f} seconds\n')


Case 1:
Time taken for DES encryption: 0.001227 seconds
Time taken for RC4 encryption: 0.000046 seconds

Case 2:
Time taken for DES encryption: 0.000088 seconds
Time taken for RC4 encryption: 0.000043 seconds

Case 3:
Time taken for DES encryption: 0.000096 seconds
Time taken for RC4 encryption: 0.000045 seconds

Case 4:
Time taken for DES encryption: 0.000054 seconds
Time taken for RC4 encryption: 0.000035 seconds



In [ ]:
# Tạo các loại dữ liệu khác nhau để làm case test
binary_data = get_random_bytes(256)  # Dữ liệu nhị phân ngẫu nhiên
numeric_data = bytes([i for i in range(256)])  # Dãy số từ 0 đến 255
repeated_data = b"A" * 256  # Dữ liệu là ký tự lặp đi lặp lại
mixed_data = b"".join([bytes([i, 65]) for i in range(128)])  # Kết hợp giữa số và chữ

# Thêm các case dữ liệu mới vào danh sách
new_cases = [
    ("Binary Data", binary_data),
    ("Numeric Data", numeric_data),
    ("Repeated Character", repeated_data),
    ("Mixed Data", mixed_data),
]

# Đo lường thời gian mã hóa cho mỗi loại dữ liệu mới
for name, data in new_cases:
    time_des = measure_time(encrypt_des, data, key_des)
    time_rc4 = measure_time(encrypt_rc4, data, key_rc4)
    print(f'{name}:')
    print(f'Time taken for DES encryption: {time_des:.6f} seconds')
    print(f'Time taken for RC4 encryption: {time_rc4:.6f} seconds\n')


Binary Data:
Time taken for DES encryption: 0.000157 seconds
Time taken for RC4 encryption: 0.000043 seconds

Numeric Data:
Time taken for DES encryption: 0.000127 seconds
Time taken for RC4 encryption: 0.000034 seconds

Repeated Character:
Time taken for DES encryption: 0.000120 seconds
Time taken for RC4 encryption: 0.000036 seconds

Mixed Data:
Time taken for DES encryption: 0.000134 seconds
Time taken for RC4 encryption: 0.000036 seconds



In [ ]:
text_content = """dfiushfodsfhd9foadjfodsfaff"""
file_path = 'small_text.txt'

with open(file_path, 'w') as f:
    f.write(text_content)

In [ ]:
import os

file_path = 'random_file.bin'
file_size = 1024 * 1024  # 1MB

with open(file_path, 'wb') as f:
    f.write(os.urandom(file_size))